In [1]:
import keras
from keras import layers, callbacks
import tensorflow as tf
import pandas as pd
import numpy as np

from keras.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

VOCAB_SIZE = 15000
SEQUENCE_LENGTH = 100
EMBED_DIM = 8

DATA_PATH=r"../../data/transformed/amazon_reviews_5_partition_1.csv"

In [2]:
# Load a subset of dataset for testing
df_reader = pd.read_csv(
    DATA_PATH,
    index_col=0,
    chunksize=10000
)
df = df_reader.__next__()

In [39]:
# load entire dataset for training
df = pd.read_csv(DATA_PATH, index_col=0)

In [40]:
df = df.dropna()
df["reviewText"] = df["reviewText"].astype("string")

In [41]:
x = df["reviewText"]
y = df["overall"] -1 # Normalize to 0-4

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [43]:
with open("../../checkpoints/vectorization_vocabulary.txt", "r") as file:
    vocab = file.read()
    vocab=vocab.split("\n")[:-1] #last line is an empty string

In [44]:
vectorize_layer = layers.TextVectorization(
    VOCAB_SIZE,
    "lower_and_strip_punctuation",
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    vocabulary=vocab
)

In [45]:
# Increase complexity

cnn_model = keras.Sequential(
    [
        layers.Input(shape=(1,), dtype="string"),
        vectorize_layer,
        layers.Embedding(VOCAB_SIZE, EMBED_DIM),
        layers.Reshape(target_shape=(100, 8, 1)),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.Conv2D(16, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(padding="same"),
        layers.Conv2D(32, 3, padding="same"),
        layers.MaxPooling2D(padding="same"),
        layers.Flatten(),
        layers.Dense(1)
    ]
)

In [46]:
cnn_model.compile(
    optimizer="adam", 
    loss="mse",
    metrics=["mae"]
)

In [47]:
cbs = [
    callbacks.ModelCheckpoint(
        filepath="../../checkpoints/2d_cnn_32_16_32_250k/model",
        save_best_only=True,
        save_weights_only=True
    ),
    callbacks.EarlyStopping(
        patience=8
    )
]

In [48]:
cnn_model.fit(
    x=x_train,
    y=y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    callbacks=cbs
)

Epoch 1/30
6431/6431 [==============================] - 432s 67ms/step - loss: 1.0578 - mae: 0.8161 - val_loss: 0.9070 - val_mae: 0.7527
Epoch 2/30
6431/6431 [==============================] - 421s 65ms/step - loss: 0.8519 - mae: 0.7262 - val_loss: 0.8726 - val_mae: 0.7324
Epoch 3/30
6431/6431 [==============================] - 421s 65ms/step - loss: 0.7764 - mae: 0.6913 - val_loss: 0.8807 - val_mae: 0.7325
Epoch 4/30
6431/6431 [==============================] - 412s 64ms/step - loss: 0.7211 - mae: 0.6648 - val_loss: 0.9004 - val_mae: 0.7501
Epoch 5/30
6431/6431 [==============================] - 433s 67ms/step - loss: 0.6769 - mae: 0.6430 - val_loss: 0.9237 - val_mae: 0.7519
Epoch 6/30
6431/6431 [==============================] - 408s 63ms/step - loss: 0.6375 - mae: 0.6232 - val_loss: 0.9350 - val_mae: 0.7568
Epoch 7/30
6431/6431 [==============================] - 412s 64ms/step - loss: 0.6044 - mae: 0.6052 - val_loss: 0.9631 - val_mae: 0.7673
Epoch 8/30
6431/6431 [===================

In [49]:
cnn_model.save("../../saved_models/2dcnn_32_16_32_250k_rows")

INFO:tensorflow:Assets written to: ../../saved_models/2dcnn_32_16_32_250k_rows\assets


INFO:tensorflow:Assets written to: ../../saved_models/2dcnn_32_16_32_250k_rows\assets


In [53]:
cnn_model.predict(["worst bad bad hated loved it so much omagahhd love love love "])

1/1 [==============================] - 0s 72ms/step


array([[2.2964468]], dtype=float32)

In [ ]:
y_train_naive = np.full_like(y_train, dtype="float32", fill_value=2.1539134783695926)

In [ ]:
mean_absolute_error(y_train, y_train_naive).numpy()

In [ ]:
predicted = lstm_model.predict(x)

In [ ]:
stats = pd.DataFrame({
    "text":x,
    "actual":y,
    "predicted":predicted.flatten()
})

In [ ]:
stats.sample(10)

In [54]:
model = keras.models.load_model('../../saved_models/2dcnn_32_16_32_250k_rows/')

In [55]:
model.predict(["tesla is an obvious scam"])

1/1 [==============================] - 0s 244ms/step


array([[0.77833277]], dtype=float32)